<a href="https://colab.research.google.com/github/Deepika4C69/Bytexl/blob/main/AI_agent_for_online_meeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub openai-whisper transformers langdetect googletrans==4.0.0-rc1 nltk
!apt install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 31.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.3 MB/s eta

In [ ]:
from pydub import AudioSegment

audio = AudioSegment.from_file("/content/meeting audio.mp3", format="mp3")
audio.export("/content/meeting audio.wav", format="wav")

<_io.BufferedRandom name='/content/meeting audio.wav'>

In [ ]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("/content/meeting audio.wav")
transcript = result["text"]
print(transcript)

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 155MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Thank you. This is the arm of Springfield meeting agenda for October the first 2024. That exactly 6 p.m. Call this meeting into order and then I'd like to get Councillor Miller for invocation place. Thank you, Mr. Mayor and by those who wish to join me in a word of prayer. Father God, we thank you for today. We thank you for that we live in a peaceful nation. We ask that you give us insight and guidance in our deliberations tonight. Lord, we want to extend the invitation to peace for Israel. We know that they're being bombarded today and it's turmoil for many people. But Lord, we lift up those who are seeking peace. We know that you're the author of peace and so we look to you for that guidance as the world is in turmoil. We pray all these things in the name of your name in your name, Lord. Amen. Thank you. Land acknowledgement, Linda. The arm of Springfield acknowledges that we are gathered on ancestral lands. Treaty one territory, traditional territory of the Anishinaabe, Kri, Oji K

In [ ]:
from transformers import pipeline
import torch

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if torch.cuda.is_available() else -1)
max_transcript_length = 1000
if len(transcript) > max_transcript_length:
    transcript = transcript[:max_transcript_length]
    print(f"Transcript truncated to {max_transcript_length} characters.")
if not transcript.strip():
    summary = "Transcript is empty or too short to summarize."
else:
    summary = summarizer(transcript, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
    print("Summary:", summary)
print("Summary:", summary)

Device set to use cpu


Summary: The arm of Springfield acknowledges that we are gathered on ancestral lands. Treaty one territory, traditional territory of the Anishinaabe, Kri, Oji K.
Summary: The arm of Springfield acknowledges that we are gathered on ancestral lands. Treaty one territory, traditional territory of the Anishinaabe, Kri, Oji K.


In [ ]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")
sentiment = sentiment_pipeline(transcript)
print("Overall Sentiment:", sentiment)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Overall Sentiment: [{'label': 'POSITIVE', 'score': 0.9992223978042603}]


In [ ]:
qa_pipeline = pipeline("question-answering")

question = "What was discussed in the meeting?"
qa_result = qa_pipeline(question=question, context=summary)
print("Q&A Result:", qa_result)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


Q&A Result: {'score': 0.2512718141078949, 'start': 77, 'end': 97, 'answer': 'Treaty one territory'}


In [ ]:
from googletrans import Translator

translator = Translator()
translated = translator.translate(summary, dest='fr')  # 'fr' for French
print("Translated Summary:", translated.text)

Translated Summary: Le bras de Springfield reconnaît que nous sommes rassemblés sur des terres ancestrales.Traité One Territory, Territoire traditionnel de l'Anishinaabe, Kri, Oji K.


In [ ]:
import smtplib
from email.message import EmailMessage

In [ ]:
SENDER_EMAIL = "akshithadantla@gmail.com"
APP_PASSWORD = "rilt qgdk fgkr nccb"

In [ ]:
def send_email(summary, recipient):
    msg = EmailMessage()
    msg.set_content(summary)
    msg["Subject"] = "Meeting Summary"
    msg["From"] = SENDER_EMAIL
    msg["To"] = recipient

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(SENDER_EMAIL, APP_PASSWORD)
            smtp.send_message(msg)
        print("Email sent successfully!")
    except Exception as e:
        print("Failed to send email:", e)

In [ ]:
summary_text = " Le bras de Springfield reconnaît que nous sommes rassemblés sur des terres ancestrales.Traité One Territory, Territoire traditionnel de l'Anishinaabe, Kri, Oji K."
recipient_email = "ru@gmail.com"  # Replace with a real email
send_email(summary_text, recipient_email)

Email sent successfully!
